<a href="https://colab.research.google.com/github/ernanhughes/rag/blob/main/sqlite_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [228]:
# Example of performing Hybrid search using sqlite-vec and FTS

In [229]:
import sqlite3
import sqlite_vec
import json
import numpy as np
import ollama


In [230]:
# Function to serialize float32 list to binary format compatible with sqlite-vec
def serialize_f32(vec):
    return np.array(vec, dtype=np.float32).tobytes()

def reciprocal_rank_fusion(fts_results, vec_results, k=60):
    rank_dict = {}

    # Process FTS results
    for rank, (id,) in enumerate(fts_results):
        if id not in rank_dict:
            rank_dict[id] = 0
        rank_dict[id] += 1 / (k + rank + 1)

    # Process vector results
    for rank, (rowid, distance) in enumerate(vec_results):
        if rowid not in rank_dict:
            rank_dict[rowid] = 0
        rank_dict[rowid] += 1 / (k + rank + 1)

    # Sort by RRF score
    sorted_results = sorted(rank_dict.items(), key=lambda x: x[1], reverse=True)
    return sorted_results

def or_words(input_string):
    # Split the input string into words
    words = input_string.split()

    # Join the words with ' OR ' in between
    result = ' OR '.join(words)

    return result

def lookup_row(id):
    row_lookup = cur.execute('''
    SELECT content FROM mango_lookup WHERE id = ?
    ''', (id,)).fetchall()
    content = ''
    for row in row_lookup:
        content= row[0]
        break
    return content


In [231]:
import requests
OLLAMA_BASE_URL="http://127.0.0.1:11434"

def generate_embeddings(text, model_name: str = "nomic-embed-text"):
    """Generate embeddings for the given text using the specified model."""
    try:
        # Send a POST request to generate embeddings
        url = f"{OLLAMA_BASE_URL}/api/embeddings"
        data = {
            "prompt": text,
            "model": model_name
        }
        response = requests.post(url, json=data)
        
        # Check if the request was successful
        if response.status_code == 200:
            return response.json().get("embedding", [])
        else:
            print(f"Failed to generate embeddings. Status code: {response.status_code}")
            print("Response:", response.text)
            return None
    
    except requests.ConnectionError:
        print("Failed to connect to the Ollama server. Make sure it is running locally and the URL is correct.")
        return None
    except json.JSONDecodeError:
        print("Failed to parse JSON response from Ollama server.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
# Example usage
# text = "Hello, world!"
# model_name = "nomic-embed-text"
# res = generate_embeddings(text, model_name)


In [232]:
# Create an in memory sqlite db
db = sqlite3.connect(":memory:")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

sqlite_version, vec_version = db.execute(
    "select sqlite_version(), vec_version()"
).fetchone()
print(f"sqlite_version={sqlite_version}, vec_version={vec_version}")


sqlite_version=3.45.3, vec_version=v0.1.3


In [233]:
data = generate_embeddings('The quick brown fox')
dims = len(data)
print ('Dims in Vector Embeddings:', dims)

Dims in Vector Embeddings: 768


In [234]:
cur = db.cursor()
cur.execute('CREATE VIRTUAL TABLE mango_fts USING fts5(id UNINDEXED, content, tokenize="porter unicode61");')

# sqlite-vec always adds an ID field
cur.execute('''CREATE VIRTUAL TABLE mango_vec USING vec0(embedding float[''' + str(dims) + '])''')

# Create a content lookup table with an index on the ID
cur.execute('CREATE TABLE mango_lookup (id INTEGER PRIMARY KEY AUTOINCREMENT, content TEXT);')

In [235]:
# Insert some sample data into mango_fts
fts_data = [
    (1, 'The quick brown fox jumps over the lazy dog.'),
    (2, 'Artificial intelligence is transforming the world.'),
    (3, 'Climate change is a pressing global issue.'),
    (4, 'The stock market fluctuates based on various factors.'),
    (5, 'Remote work has become more prevalent during the pandemic.'),
    (6, 'Electric vehicles are becoming more popular.'),
    (7, 'Quantum computing has the potential to revolutionize technology.'),
    (8, 'Healthcare innovation is critical for societal well-being.'),
    (9, 'Space exploration expands our understanding of the universe.'),
    (10, 'Cybersecurity threats are evolving and becoming more sophisticated.')
]

cur.executemany('''
INSERT INTO mango_fts (id, content) VALUES (?, ?)
''', fts_data);


cur.executemany('''
  INSERT INTO mango_lookup (id, content) VALUES (?, ?)
''', fts_data)


# Generate embeddings for the content and insert into mango_vec
for row in fts_data:
    id, content = row
    embedding = generate_embeddings(content)
    cur.execute('''
    INSERT INTO mango_vec (rowid, embedding) VALUES (?, ?)
    ''', (id, serialize_f32(list(embedding))))


# Commit changes
db.commit()

In [236]:
def search(fts_search_query: str = "Electric", top_k: int = 2):
    fts_results = cur.execute('''
    SELECT id FROM mango_fts WHERE mango_fts MATCH ? ORDER BY rank limit 5
    ''', (or_words(fts_search_query),)).fetchall()

    # Vector search query
    query_embedding = generate_embeddings(fts_search_query)
    vec_results = cur.execute('''
    SELECT rowid, distance FROM mango_vec WHERE embedding MATCH ? and K = ?
    ORDER BY distance
    ''', [serialize_f32(list(query_embedding)), top_k]).fetchall()

    # Combine results using RRF
    combined_results = reciprocal_rank_fusion(fts_results, vec_results)

    # Print combined results
    for id, score in combined_results:
        print(f'ID: {id}, Content: {lookup_row(id)}, RRF Score: {score}')    


In [237]:
print("---- technology ----")
search("technology")
print("---- Electric ----")
search("Electric")  
print("---- Medical ----")
search("medical")


---- technology ----
ID: 7, Content: Quantum computing has the potential to revolutionize technology., RRF Score: 0.03278688524590164
ID: 2, Content: Artificial intelligence is transforming the world., RRF Score: 0.016129032258064516
---- Electric ----
ID: 6, Content: Electric vehicles are becoming more popular., RRF Score: 0.03278688524590164
ID: 7, Content: Quantum computing has the potential to revolutionize technology., RRF Score: 0.016129032258064516
---- Medical ----
ID: 8, Content: Healthcare innovation is critical for societal well-being., RRF Score: 0.01639344262295082
ID: 5, Content: Remote work has become more prevalent during the pandemic., RRF Score: 0.016129032258064516


In [238]:
# Close the connection
db.close()
